In [ ]:
import numpy as np
import pandas as pd
from dynamicslib.common import *
from dynamicslib.consts import *
from dynamicslib.continuation import *
from dynamicslib.targeter import *
from dynamicslib.plotting import *

In [ ]:
# X = [y, vx, vy, tf]
# f = [y-y0, vx-vx0, vy-vy0]


def xtf2X(x0, tf):
    return np.array([x0[1], x0[-3], x0[-2], tf])


def X2xtf(X):
    return np.array([1 / 2 - muEM, X[0], 0, X[1], X[2], 0]), X[3]


def stmeom2DF(eomf, stm):
    dF = np.array(
        [
            [stm[1, 1] - 1, stm[1, -3], stm[1, -2], eomf[1]],
            [stm[-3, 1], stm[-3, -3] - 1, stm[-3, -2], eomf[-3]],
            [stm[-2, 1], stm[-2, -3], stm[-2, -2] - 1, eomf[-2]],
        ]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1] - x0[1], xf[-3] - x0[-3], xf[-2] - x0[-2]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, True, muEM, 1e-12)

In [ ]:
def get_IC(short=True, mu=muEM, beta1=-1e-3):
    g = 1 - 27 * mu * (1 - mu)
    s2 = np.abs(np.sqrt(1 / 2 * (-1 - np.sqrt(g)), dtype=np.complex128))  # SP
    s1 = np.abs(np.sqrt(1 / 2 * (-1 + np.sqrt(g)), dtype=np.complex128))  # LP
    sterm = s2 if short else s1
    Uxx = 3 / 4
    Uxy = 3 * np.sqrt(3) / 2 * (1 / 2 - mu)
    Uyy = 9 / 4
    mtx = np.array([[Uxx + sterm**2, Uxy], [Uxy, Uyy + sterm**2]])
    t1, t2 = mtx @ np.array([0, beta1])
    beta2 = t1 / (2 * sterm)
    alpha2 = t2 / (-2 * sterm)
    y0 = beta1
    vx0 = -sterm * alpha2
    vy0 = -sterm * beta2
    period = 2 * np.pi / sterm

    sv0 = np.array([1 / 2 - mu, y0 + np.sqrt(3) / 2, 0, vx0, vy0, 0])
    return xtf2X(sv0, period)

In [ ]:
XsSP, eSP = arclen_cont(
    get_IC(True), func, [-1, 0, 0, 0], 5e-3, 10, 1e-10, modified=True
)

In [ ]:
XsLP, eLP = arclen_cont(
    get_IC(False), func, [-1, 0, 0, 0], 5e-3, 8, 1e-10, modified=True
)

In [ ]:
es = [eLP, eSP]
XsAll = [XsLP, XsSP]

param_names = [
    "Index",
    "Initial x",
    "Initial y",
    "Initial vx",
    "Initial vy",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


for Lp in range(2):
    xyzs_lp = []
    data_lp = []
    iterator = list(enumerate(zip(XsAll[Lp], es[Lp])))
    for i, (X, evals) in tqdm(iterator):
        Xcp = X.copy()
        xyzs_lp.append(prop_ic(Xcp, X2xtf))
        args = np.argsort(np.abs(evals))
        # eigs = evals[[*args[:2], *args[-2:]]]
        stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
        jc, period = get_JC_tf(Xcp, X2xtf)
        data_lp.append([i, 1 / 2 - muEM, *Xcp[:-1], period, jc, stabind, *evals])
    xyzs.append(xyzs_lp)
    data.append(data_lp)

In [ ]:
# df = pd.DataFrame(data[1], columns=param_names).set_index("Index")
# df.to_csv("L4 Short Period.csv")

In [ ]:
# df = pd.DataFrame(data[0], columns=param_names).set_index("Index")
# df.to_csv("L4 Long Period.csv")